# PHASE 1: DATA PROCESSING

In [1]:
import pandas as pd
import numpy as np
import datetime 
import time 
from datetime import datetime

### 1. DATA LOAD

Charge 6 dataframes with information about last three months of 2020, origin Madrid/Barcelona, through the path "../data", with categoric and numeric variables. dfdistancia file is also loaded

In [2]:
df_ryanair = pd.read_csv("../data/data_ryanair.csv")
df_vbaratos = pd.read_csv("../data/data_vuelosbaratos.csv", dtype={'PRECIO': object})
df_vbaratos1 = pd.read_csv("../data/data_vuelosbaratosPortatil.csv", dtype={'PRECIO': object})
df_vbaratos2 = pd.read_csv("../data/data_vuelosbaratosSobremesa.csv", dtype={'PRECIO': object})
df_vbaratos3 = pd.read_csv("../data/data_vuelosbaratos1.csv", dtype={'PRECIO': object})
df_vbaratos4 = pd.read_csv("../data/data_vuelosbaratos_Sobremesa.csv", dtype={'PRECIO': object})
df_distancia = pd.read_csv("../data/dfdistancia.csv")

In order to manage data in a single dataframe, data have been unified

In [3]:
df_vbaratos['PRECIO'] = df_vbaratos['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos1['PRECIO'] = df_vbaratos1['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos2['PRECIO'] = df_vbaratos2['PRECIO'].apply(lambda x: x.replace('.', ',').replace(',', '')).astype('float')
df_vbaratos3['PRECIO'] = df_vbaratos3['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos4['PRECIO'] = df_vbaratos4['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')

In [4]:
df = pd.concat([df_ryanair, df_vbaratos, df_vbaratos1, df_vbaratos2, df_vbaratos3, df_vbaratos4 ])

In [5]:
df.shape

(2661808, 12)

### 2. INITIAL DATA EXPLORATION

#### Information Observation

In [6]:
#Función de la librería para que muestre todas las columnas
pd.options.display.max_columns = None

In [7]:
df.head()

,Unnamed: 0,CAPTURA,FUENTE,ORIGEN,DESTINO,SALIDA_IDA,LLEGADA_IDA,SALIDA_VUELTA,LLEGADA_VUELTA,PRECIO,URL,AEROLINEA
0,0,2020-09-09,Ryanair,BCN,AGP,2020-10-01 09:25,2020-10-01 11:20,2020-10-08 07:15,2020-10-08 08:50,29.98,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair
1,1,2020-09-09,Ryanair,BCN,BGY,2020-10-01 20:15,2020-10-01 22:00,2020-10-08 22:25,2020-10-08 23:59,25.98,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair
2,2,2020-09-09,Ryanair,BCN,BHX,2020-10-01 12:40,2020-10-01 14:10,2020-10-08 14:35,2020-10-08 17:45,29.61,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair
3,3,2020-09-09,Ryanair,BCN,BRU,2020-10-01 06:05,2020-10-01 08:20,2020-10-08 08:55,2020-10-08 11:00,30.24,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair
4,4,2020-09-09,Ryanair,BCN,BUD,2020-10-01 17:00,2020-10-01 19:40,2020-10-08 20:05,2020-10-08 22:40,31.24,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair


In [8]:
#Función para enlistar todas las columnas y poder visualizarlas con mayor facilidad.
df.columns.to_list()

['Unnamed: 0',
 'CAPTURA',
 'FUENTE',
 'ORIGEN',
 'DESTINO',
 'SALIDA_IDA',
 'LLEGADA_IDA',
 'SALIDA_VUELTA',
 'LLEGADA_VUELTA',
 'PRECIO',
 'URL',
 'AEROLINEA']

After the first observation, the treatment of the columns DEPARTURE_GOING, ARRIVAL_GOING, DEPARTURE_RETURN and ARRIVAL_RETURN has been carried out. For a better analysis it will be convenient to divide these columns into dates and times that indicate this information separately.

In [9]:
#División de una cadena o string a través de la función split.
df[['fecha_salida_ida','hora_salida_ida']] = df.SALIDA_IDA.str.split(expand=True)
df[['fecha_llegada_ida','hora_llegada_ida']] = df.LLEGADA_IDA.str.split(expand=True)
df[['fecha_salida_vuelta','hora_salida_vuelta']] = df.SALIDA_VUELTA.str.split(expand=True)
df[['fecha_llegada_vuelta','hora_llegada_vuelta']] = df.LLEGADA_VUELTA.str.split(expand=True)

#### Checking for duplicated values

To ensure the reliability of the dataset, since the data has been extracted by several people in different processes, a previous control will be made based on the redundancy of information by duplicated records. Duplicate values are eliminated.

In [10]:
print(df.shape)
df.drop_duplicates(keep='first', inplace=True)
print(df.shape)
df.head()

(2661808, 20)
(2484667, 20)


,Unnamed: 0,CAPTURA,FUENTE,ORIGEN,DESTINO,SALIDA_IDA,LLEGADA_IDA,SALIDA_VUELTA,LLEGADA_VUELTA,PRECIO,URL,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta
0,0,2020-09-09,Ryanair,BCN,AGP,2020-10-01 09:25,2020-10-01 11:20,2020-10-08 07:15,2020-10-08 08:50,29.98,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair,2020-10-01,09:25,2020-10-01,11:20,2020-10-08,07:15,2020-10-08,08:50
1,1,2020-09-09,Ryanair,BCN,BGY,2020-10-01 20:15,2020-10-01 22:00,2020-10-08 22:25,2020-10-08 23:59,25.98,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair,2020-10-01,20:15,2020-10-01,22:00,2020-10-08,22:25,2020-10-08,23:59
2,2,2020-09-09,Ryanair,BCN,BHX,2020-10-01 12:40,2020-10-01 14:10,2020-10-08 14:35,2020-10-08 17:45,29.61,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair,2020-10-01,12:40,2020-10-01,14:10,2020-10-08,14:35,2020-10-08,17:45
3,3,2020-09-09,Ryanair,BCN,BRU,2020-10-01 06:05,2020-10-01 08:20,2020-10-08 08:55,2020-10-08 11:00,30.24,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair,2020-10-01,06:05,2020-10-01,08:20,2020-10-08,08:55,2020-10-08,11:00
4,4,2020-09-09,Ryanair,BCN,BUD,2020-10-01 17:00,2020-10-01 19:40,2020-10-08 20:05,2020-10-08 22:40,31.24,https://www.ryanair.com/es/es/trip/flights/sel...,Ryanair,2020-10-01,17:00,2020-10-01,19:40,2020-10-08,20:05,2020-10-08,22:40


As you can see, all duplicate values have been eliminated

#### Comprobación de la existencia de valores nulos

An exploration of those null data that may occur in the dataset will be carried out and if so, it will be assessed later on how to treat them.

In [11]:
df.isnull().sum()

Unnamed: 0               0
CAPTURA                  0
FUENTE                   0
ORIGEN                   0
DESTINO                  0
SALIDA_IDA               0
LLEGADA_IDA              0
SALIDA_VUELTA            0
LLEGADA_VUELTA           0
PRECIO                   0
URL                      0
AEROLINEA               33
fecha_salida_ida         0
hora_salida_ida          0
fecha_llegada_ida        0
hora_llegada_ida         0
fecha_salida_vuelta      0
hora_salida_vuelta       0
fecha_llegada_vuelta     0
hora_llegada_vuelta      0
dtype: int64

The existence of 33 null values in the AIRLINE column is checked.

## 3. DATA CLEANING

At this point we can co-prove that it is convenient to eliminate at least those columns that we already know can add noise to the future analysis. In this case it will be the Unnamed columns: 0 the date columns that have been transformed and the URL column.

In [12]:
df.drop(['Unnamed: 0', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA', 'URL'], axis='columns', inplace=True)

The null values are analyzed and it is decided to eliminate the 33 rows because they are insignificant compared to the amount of information handled.

In [13]:
df[df['AEROLINEA'].isnull()]

,CAPTURA,FUENTE,ORIGEN,DESTINO,PRECIO,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta
243646,2020-09-11,vuelosbaratos,BCN,RTM,619.0,NaN,2020-10-10,19:50,2020-10-11,16:00,2020-10-17,17:30,2020-10-18,14:05
243648,2020-09-11,vuelosbaratos,BCN,RTM,619.0,NaN,2020-10-10,18:20,2020-10-11,16:00,2020-10-17,17:30,2020-10-18,14:05
243650,2020-09-11,vuelosbaratos,BCN,RTM,619.0,NaN,2020-10-10,14:50,2020-10-11,16:00,2020-10-17,17:30,2020-10-18,14:05
243652,2020-09-11,vuelosbaratos,BCN,RTM,619.0,NaN,2020-10-10,13:35,2020-10-11,16:00,2020-10-17,17:30,2020-10-18,14:05
243657,2020-09-11,vuelosbaratos,BCN,RTM,621.0,NaN,2020-10-10,19:50,2020-10-11,16:00,2020-10-17,18:00,2020-10-18,14:05
243659,2020-09-11,vuelosbaratos,BCN,RTM,621.0,NaN,2020-10-10,18:20,2020-10-11,16:00,2020-10-17,18:00,2020-10-18,14:05
243661,2020-09-11,vuelosbaratos,BCN,RTM,621.0,NaN,2020-10-10,14:50,2020-10-11,16:00,2020-10-17,18:00,2020-10-18,14:05
243663,2020-09-11,vuelosbaratos,BCN,RTM,621.0,NaN,2020-10-10,13:35,2020-10-11,16:00,2020-10-17,18:00,2020-10-18,14:05
243690,2020-09-11,vuelosbaratos,BCN,RTM,864.0,NaN,2020-10-10,08:55,2020-10-10,21:30,2020-10-17,17:30,2020-10-18,14:05
243691,2020-09-11,vuelosbaratos,BCN,RTM,864.0,NaN,2020-10-10,08:55,2020-10-10,21:45,2020-10-17,17:30,2020-10-18,14:05


In [14]:
#Function to delete rows containing at least one NaN
print(df.shape)
df.dropna(axis=0, inplace = True) 
print(df.shape)

(2484667, 14)
(2484634, 14)


## 4. DATASET ENRICHTMENT

The column with the distances between airports as a variable to be studied is created

In [15]:
df_distancia = df_distancia.iloc[:,2:].values
df['distancia']= df_distancia

Another column is added about holidays that can provide extra data

In [16]:
festivos_mad = ['2020-10-12', '2020-11-02', '2020-11-09', '2020-12-07', '2020-12-08', '2020-12-25', '2021-01-01', '2021-01-06']
festivos_bcn = ['2020-10-12', '2020-12-08', '2020-12-25', '2020-12-26', '2021-01-01', '2021-01-06']

In [17]:
df['FESTIVO']=False

In [18]:
def esFestivo(origen, fecha):
    if (origen == 'MAD') & (fecha in festivos_mad):
        return True
    elif (origen == 'BCN') & (fecha in festivos_bcn):
        return True
    else:
        return False
    
df['FESTIVO'] = df.apply(lambda x: esFestivo(x.ORIGEN, x.fecha_salida_ida), axis=1)

In [19]:
df

,CAPTURA,FUENTE,ORIGEN,DESTINO,PRECIO,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta,distancia,FESTIVO
0,2020-09-09,Ryanair,BCN,AGP,29.98,Ryanair,2020-10-01,09:25,2020-10-01,11:20,2020-10-08,07:15,2020-10-08,08:50,766.443808,False
1,2020-09-09,Ryanair,BCN,BGY,25.98,Ryanair,2020-10-01,20:15,2020-10-01,22:00,2020-10-08,22:25,2020-10-08,23:59,784.895263,False
2,2020-09-09,Ryanair,BCN,BHX,29.61,Ryanair,2020-10-01,12:40,2020-10-01,14:10,2020-10-08,14:35,2020-10-08,17:45,1273.647883,False
3,2020-09-09,Ryanair,BCN,BRU,30.24,Ryanair,2020-10-01,06:05,2020-10-01,08:20,2020-10-08,08:55,2020-10-08,11:00,1083.476587,False
4,2020-09-09,Ryanair,BCN,BUD,31.24,Ryanair,2020-10-01,17:00,2020-10-01,19:40,2020-10-08,20:05,2020-10-08,22:40,1525.338415,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443627,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,09:05,2020-12-29,08:15,2021-01-04,10:45,2021-01-04,21:55,1845.621168,False
443628,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,09:05,2020-12-29,10:15,2021-01-04,10:45,2021-01-04,21:55,1845.621168,False
443629,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,07:25,2020-12-28,16:30,2021-01-04,13:00,2021-01-05,18:50,1845.621168,False
443630,2020-09-13,vuelosbaratos,MAD,SPC,330.00,Vueling,2020-12-28,07:45,2020-12-28,19:45,2021-01-04,13:00,2021-01-05,09:40,1845.621168,False


We also added a new column with the day of the week

In [20]:
df['dia_semana'] = df['fecha_salida_ida'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").weekday())

Once this is done, the columns are converted from date to datetime format.

In [21]:
for i in df.columns[6:13:2]:
    df[i] = df[i].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df['CAPTURA'] = df['CAPTURA'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

We rounded off the price and distance columns

In [22]:
df["distancia"] = df["distancia"].apply(lambda x:round(x, 2))
df["PRECIO"] = df["PRECIO"].apply(lambda x:round(x, 2))

We created the Euro column per hectokilometer to have a more exact precision when comparing prices.

In [23]:
df['hkmEUR'] = df.apply(lambda x: round((x.PRECIO/x.distancia)*100,2), axis=1)

In [24]:
df.head()

,CAPTURA,FUENTE,ORIGEN,DESTINO,PRECIO,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta,distancia,FESTIVO,dia_semana,hkmEUR
0,2020-09-09,Ryanair,BCN,AGP,29.98,Ryanair,2020-10-01,09:25,2020-10-01,11:20,2020-10-08,07:15,2020-10-08,08:50,766.44,False,3,3.91
1,2020-09-09,Ryanair,BCN,BGY,25.98,Ryanair,2020-10-01,20:15,2020-10-01,22:00,2020-10-08,22:25,2020-10-08,23:59,784.90,False,3,3.31
2,2020-09-09,Ryanair,BCN,BHX,29.61,Ryanair,2020-10-01,12:40,2020-10-01,14:10,2020-10-08,14:35,2020-10-08,17:45,1273.65,False,3,2.32
3,2020-09-09,Ryanair,BCN,BRU,30.24,Ryanair,2020-10-01,06:05,2020-10-01,08:20,2020-10-08,08:55,2020-10-08,11:00,1083.48,False,3,2.79
4,2020-09-09,Ryanair,BCN,BUD,31.24,Ryanair,2020-10-01,17:00,2020-10-01,19:40,2020-10-08,20:05,2020-10-08,22:40,1525.34,False,3,2.05


## 5. DATA EXPORT

Finally the data will be exported in a csv with the name of data_vuelos_completo.csv to the data folder in the project repository. Relative path "../../data".

In [25]:
df.to_csv("../data/data_vuelos_completo.csv", index=False)